In [49]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import os
import re
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
# data_utils.download_data_gdown("./")

In [11]:
%%time
ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

CPU times: user 7.58 s, sys: 1.01 s, total: 8.59 s
Wall time: 8.85 s


In [47]:
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [50]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
df

,article_id,text
0,0,醫師：最近人有沒有什麼不舒服？民眾：沒有。醫師：沒有喔。我們本來說要月，年底才打對不對？民眾...
1,1,醫師：要去上課嗎？民眾：沒有。這學期已經開始不上課了。醫師：啊今天的，呃，應該是昨天的抽血。...
2,2,醫師：後來那個吃藥沒有問題對不對？民眾：沒有。醫師：诶？你是打針的？民眾：對。醫師：阿身上有...
3,3,民眾：我自己昨天開始，一天吃一次一粒。醫師：一次一粒，減半量還好？民眾：阿可是……減到今天早...
4,4,醫師：喔，啊嘉明最近還好嗎？民眾：ＯＫ。醫師：啊今年的流感打了嗎？民眾：還沒。醫師：那我們今...
...,...,...
154,154,醫師：這個月還好嗎？民眾：就……醫師：這個月有去碰嗎？民眾：沒有，不去碰的時候就會覺得慾望非...
155,155,醫師：啊這個月還好嗎？民眾：還好啊。醫師：好，所以你反正天天吃啦齁？民眾：對。醫師：好，我們...
156,156,醫師：好那所以總之就是好，所以總之反正，總之就是……這這……就是反正你剩下的藥你就是……就是...
157,157,醫師：好，所以第一個月吃藥的狀況怎麼樣？民眾：嗯，吃藥狀況喔？醫師：嗯。民眾：就是吃了，一開...


In [ ]:
sentence_list = df['text'].to_list()

In [25]:
article = '醫師：你有做超音波嘛，那我們來看報告，有些部分有紅字耶。民眾：紅字是甚麼意思？醫師：就是肝功能有比較高，肝功能68，就是這個ALP是68，這樣比較高，正常應是50以下，另外就是你之前說你有B肝，但是你B肝已經好了耶。民眾：它會自動修復阿。醫師：你有抗體了阿，所以你B肝已經沒帶原了耶。民眾：我以前被關的時候，就有在固定驗血，那時候說有B肝。醫師：阿你現在已經有保護的抗體了。但是你現在有C肝。民眾：C肝？醫師：C型肝炎，A、B、C的C型肝炎。這部分你沒有檢查過對不對。民眾：沒有耶，但是一般被關的時候都會驗血，還是他只會驗B肝沒有驗C肝？醫師：一般都會這樣耶。民眾：但是那時候他只跟我說B肝，沒C肝。醫師：那你那是甚麼時候做的檢查？民眾：很久以前了。醫師：我建議這個C肝要檢查一下。民眾：要怎麼檢查？要抽血嗎？醫師：要再做進一步更精密的檢查，看C型肝炎的基因，這樣比較準。然後另外就是我建議你做是因為現在C肝目前有藥物可以治療。民眾：恩對。醫師：你知道嘛，C肝因為有符合歐洲的標準，政府會幫你出錢。民眾：喔～醫師：可以申請看看。民眾：喔～醫師：阿這個是意外發現啦。民眾：喔～醫師：對阿，所以我們就會另外看一下。也許已經好了，假如在測病毒的基因，假如測起來好了，那也沒關係，但你要確定，要再做進一步的檢查確定這樣子。民眾：喔～醫師：阿超音波的話，你說你都是腰側右側這邊會痛嗎？民眾：恩，這邊會痛。他這個有可能是因為我坐骨神經痛，會不會是因為我久沒去復健，才會這樣？醫師：一般都是如果有受傷過或是做工作的話，有受傷過的話，那個脊椎尾骨壓到神經。民眾：喔～那會引起這樣狀況嗎？醫師：他會繞一圈拉，他如果是神經的話。民眾：對，他通常都從後面腰椎右側這邊開始痛起來。醫師：這樣喔。民眾：想說所以真的會這樣影響到嗎？醫師：這邊是坐骨神經耶，上面這邊是脊椎耶。民眾：不是，我本來受傷的時候我走路都會不太能走，然後去復健的時候都會拉開，拉開才比較會好。然後我現在手還會痛。醫師：你腰椎痛跟手痛會有關係嗎？民眾：不知道耶。醫師：因為你這邊是腰椎耶，受傷比較多，然後上面這個地方是胸椎耶。民眾：喔，算是不同這樣就對了喔。醫師：對，那所以胸椎這邊會痛嗎，你會嗎？民眾：不會耶。就是腰椎這邊會刺痛，有時候會，有時候不會，抽痛抽痛這樣。醫師：那看一下超音波影像左邊，這裡是肝臟，然後探頭是放肚子中間這邊。民眾：恩。醫師：看起來也還好，看起來沒有長甚麼。那看其他超音波影像，這邊有看到肌肉，下面這邊就是肝臟，下面旁邊是膽囊，膽囊的地方在這裡。民眾：喔～醫師：所以肝、膽、腎臟應該有照給你看，就是貼在肝臟的下面。民眾：恩。醫師：你有一顆腎臟水瘤。民眾：恩。醫師：這顆，是一個水泡，這是良性的。民眾：喔～這樣就看的出來了喔？醫師：超音波影像這裡看起來就不同，水的話就會看起來就會是黑黑的。民眾：恩。醫師：因為腎臟有水泡，他本來就是過濾的器官，所以長水泡在發育上有的時候就會這樣。民眾：喔～醫師：但是左邊的腎臟反而沒怎樣。然後脾臟在旁邊，在左邊這邊，脾臟的大小是10.78公分，這樣是正常，所以跟脾臟沒關係。民眾：恩。醫師：所以你這些器官看起來是還好。那另外一個器官就是胃，你的胃會怎麼樣嗎？胃會脹脹的還是怎麼樣嗎？民眾：恩，會胃脹脹的。但是我有去過三多路上的胃腸科診所看過，也是給長庚醫院醫師照過，他說是甚麼我也忘記了。醫師：是胃潰瘍嗎？民眾：他說是胃最上面這邊，他說是胃酸最多的地方，他也有幫我照過了，他說沒什麼。醫師：沒怎樣這樣嘛。民眾：我有先去他那邊照，再來這邊。醫師：喔，所以沒有胃潰瘍，所以應該是還好。民眾：這種好像都要吞胃鏡最準，對嗎？醫師：如果要看胃的話，就要照胃鏡去看。如果是胃炎就要看表面啦。民眾：恩。醫師：這個超音波的話是看實體器官，實心的器官啦。但是因為胃是袋狀的器官，所以要用胃鏡去看。民眾：恩。但是我手痛這邊好像不是胃。像是肌肉扭到或是肌肉瘀青，如果拉到或是碰到會痛。醫師：痛會牽到後面嗎？民眾：不會。醫師：只有這個地方會痛而已嗎？民眾：對，就只有這個點會痛而已。醫師：但是你脊椎還沒照過超音波嘛。但是你腰椎這邊是會痠嘛。民眾：對，以前有傷到過，不是撞到，應該是下車的時候，從比較矮的車子出來的時候，要彎下腰出來，就扭到，就是要下車的時候，左腳跨出來，要彎下腰再直起來的時候就會扭到。醫師：這是不是還要再幫你照一下超音波？民眾：應該是不用再照了啦，之前有照過了。醫師：有照過這樣好啦。民眾：重點應該是要復健，但是我很懶得去復健啦。我十天沒去了。醫師：但現在最重要的應該是你的肝的部分，C肝要檢查。民眾：如果沒睡的話，這個指數會升高嗎？醫師：會，如果喝酒、沒睡、工作過度、沒有休息的話，就會這樣子，就指數會比較高一點。民眾：喔～醫師：那我今天就幫你抽好了，驗一下C肝的基因這樣子。民眾：好啊。醫師：然後我開個保肝片給你好不好？民眾：好啊。醫師：好齁。民眾：結果還是要很努力去做復健嘛。這樣還是要看時間去。醫師：對阿，因為還要工作，你做甚麼工作？民眾：我打撲克牌。醫師：打撲克牌？你說真的嗎？民眾：真的阿。醫師：是。民眾：我打撲克牌都會坐整天。醫師：是電腦的那種撲克牌嗎？民眾：不是啦，是人跟人在打的那種。醫師：你開賭場喔？民眾：沒有啦，我們都是朋友約一約，然後跟比較好的老闆約一下來打而已。醫師：沒有，開個玩笑一下而已這樣。民眾：喔～沒有啦。醫師：因為我有個病人是開賭場，他說他很忙。民眾：這樣喔。醫師：那假如真的還有病毒的複合量，就代表體內還有C肝的病毒。我再拜託我同事幫你處理，他可以申請。民眾：因為我以前有吸過海洛英。醫師：哇～民眾：很久以前了。就是因為注射那個才引起肝炎。醫師：對阿，因為大部分C肝都是因為打針引起的。阿你HIV有驗過嗎？民眾：有阿，以前被關的時候都有驗過。醫師：不然等一下也可以順便驗一下，因為我們驗C肝的時候都會順便驗一下。民眾，好啊。醫師：那我們就順便驗一下。民眾：好啊。醫師：但是我們驗這個不用空腹這樣子，等一下就可以直接去抽血了。民眾：好啊。醫師：然後我們之後抽完，回來看一下報告。民眾：報告這麼快出來喔？醫師：對。沒辦法馬上看，因為這個比較精密。民眾：喔，改天來看。醫師：對，改天來看。民眾：好啊，都可以。醫師：這差不多要四五天的作業。民眾：好啊，都可以。醫師：可以啦齁，那我們就3月18號。民眾：下禮拜好了。醫師：下禮拜嗎？那我們3月24日看。民眾：那天假日耶。護理師：3月24醫院有看診。民眾：禮拜二到禮拜四其中一天好了，不要假日，假日沒空。醫師：不然3月31日好了？民眾：好啊。醫師：沒差嘛。民眾：恩，沒關係。因為這個是慢性病比較沒差嘛。醫師：對，他沒有立即性的危險，但是要追蹤啦。民眾：喔～因為C肝現在吃藥都要看病一下。醫師：你有朋友在治療的嗎？民眾：有阿，他都自己自費，好像很貴。醫師：有一些病人會去印度買學名藥，因為印度仿的，效果不錯，但是比原廠藥還要便宜。民眾：好。醫師：那外面稍等一下，等一下給你單子去抽血。民眾：好啊。'
# sentence_list = article.split('。')
# sentence_list = re.split('\uff0c|\u3002|\uff1f', article)
sentence_list = [article]

In [55]:
# word_sentence_list = ws(
#     sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
# )

In [56]:
%%time
word_sentence_list = ws(sentence_list)
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [61]:
def get_entity_type(entity_type):
    if entity_type == 'DATE':
        return 'time'
    if entity_type == 'TIME':
        return 'time'
    if entity_type == 'CARDINAL':
        return 'med_exam'
    if entity_type == 'QUANTITY':
        return 'med_exam'
    if entity_type == 'LOC':
        return 'location'
    if entity_type == 'FAC':
        return 'location'
    if entity_type == 'ORG':
        return 'location'
    if entity_type == 'GPE':
        return 'location'    
    if entity_type == 'MONEY':
        return 'money'    

In [62]:
c = 0
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
#     for word, pos in zip(word_sentence, pos_sentence):
#         print(f"{word}({pos})", end="\u3000")
#     print()
    return
    
for i, sentence in enumerate(sentence_list):
#     print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        start_position = entity[0]
        end_position = entity[1]
        entity_text = entity[3]
        entity_type = get_entity_type(entity[2])
        if entity_type:
            row = '{}\t{}\t{}\t{}'.format(start_position, end_position, entity_text, entity_type)
            print(row)
        else:
            print(entity)

36	38	年底	time
113	115	今天	time
227	229	麻豆	location
237	239	麻豆	location
295	297	今年	time
455	457	年底	time
484	486	年底	time
15	18	這學期	time
31	33	今天	time
40	42	昨天	time
76	80	0.79	med_exam
94	96	88	med_exam
111	113	60	med_exam
122	124	60	med_exam
179	181	20	med_exam
182	184	19	med_exam
258	261	2.6	med_exam
(360, 361, 'PERSON', '黃')
366	369	6.6	med_exam
385	388	6.6	med_exam
394	397	8.3	med_exam
407	410	8.3	med_exam
414	417	四個月	time
418	421	8.3	med_exam
463	466	6.2	med_exam
469	472	三個月	time
(497, 498, 'PERSON', '劉')
(626, 627, 'PERSON', '黃')
(663, 666, 'PERSON', '格里曼')
(670, 673, 'PERSON', '格里曼')
(720, 723, 'PERSON', '格里曼')
742	744	中午	time
(747, 750, 'PERSON', '格里曼')
(754, 757, 'PERSON', '格里曼')
(811, 813, 'PERSON', '費德')
867	869	21	med_exam
873	875	21	med_exam
881	884	12天	time
890	893	12天	time
902	906	三個星期	time
(996, 998, 'ORDINAL', '第一')
168	169	4	med_exam
245	249	1145	med_exam
275	278	500	med_exam
285	288	500	med_exam
390	393	三個月	time
402	405	三個月	time
513	515	一半	med_exam
522	524	一半	med_exam
67

198	202	兩個禮拜	time
205	208	一個月	time
364	366	五月	time
435	437	高雄	location
438	440	台南	location
440	441	兩	med_exam
601	603	開封	location
(632, 633, 'PERSON', '王')
650	651	三	med_exam
18	22	兩個多月	time
142	145	兩個月	time
(150, 152, 'PERSON', '恩亨')
168	169	一	med_exam
176	177	一	med_exam
283	287	兩個禮拜	time
290	291	一	med_exam
306	308	新竹	location
326	328	新竹	location
333	335	鹿港	location
369	371	鹿港	location
373	375	新竹	location
391	393	半年	time
402	404	今天	time
405	412	二十三號到七號	time
419	421	今天	time
446	447	兩	med_exam
455	458	前幾天	time
462	465	前幾天	time
(465, 466, 'PERSON', '陳')
538	541	上個月	time
(556, 557, 'PERSON', '阿')
557	560	前幾天	time
(560, 561, 'PERSON', '陳')
579	581	30	med_exam
(597, 599, 'PERSON', '阿，')
(705, 708, 'PERSON', '阿反正')
714	716	30	med_exam
(725, 726, 'PERSON', '恩')
902	905	一個月	time
905	906	一	med_exam
909	913	一個禮拜	time
913	914	一	med_exam
919	923	一個禮拜	time
923	924	一	med_exam
929	933	一個禮拜	time
947	950	兩個月	time
1082	1084	今年	time
1131	1132	兩	med_exam
1182	1183	二	med_exam
1192	1197	四百一十四	med_exam
1199	